In [26]:
import requests
import pandas as pd

lat, lon = 52.52, 13.41

params = (
    "WS10M,WD10M,RH2M,QV2M,PS,T2MDEW,T2MWET,PRECTOTCORR,"
    "ALLSKY_KT,ALLSKY_SRF_ALB,SZA,ALLSKY_SFC_UV_INDEX,ALLSKY_SFC_PAR_TOT"
)

def fetch_range(start, end):
    url = (
        "https://power.larc.nasa.gov/api/temporal/hourly/point?"
        f"parameters={params}&community=RE"
        f"&longitude={lon}&latitude={lat}"
        f"&start={start}&end={end}&format=JSON"
    )
    r = requests.get(url)
    r.raise_for_status()
    d = r.json()
    df_ = pd.DataFrame(d["properties"]["parameter"]).transpose().reset_index()
    df_.rename(columns={"index": "variable"}, inplace=True)
    return df_

dfs = []
dfs.append(fetch_range("20230101", "20231231"))
dfs.append(fetch_range("20240101", "20241231"))
dfs.append(fetch_range("20250101", "20250731"))
# WIDE format
df_all = pd.concat(dfs, ignore_index=True)
df_all = df_all.sort_values("variable").drop_duplicates(subset="variable").reset_index(drop=True)




In [28]:
df_all.rename(columns={'datetime': 'variable'})

,variable,2023010100,2023010101,2023010102,2023010103,2023010104,2023010105,2023010106,2023010107,2023010108,...,2025073114,2025073115,2025073116,2025073117,2025073118,2025073119,2025073120,2025073121,2025073122,2025073123
0,ALLSKY_KT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ALLSKY_SFC_PAR_TOT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00
2,ALLSKY_SFC_UV_INDEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ALLSKY_SRF_ALB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PRECTOTCORR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,PS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,QV2M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.40,9.55,9.72,9.93,10.22,10.26,10.00,9.85,9.80,9.81
7,RH2M,87.9,87.59,88.64,89.48,89.08,87.74,86.75,87.31,88.84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,SZA,90.0,90.00,90.00,90.00,90.00,90.00,90.00,90.00,89.02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,T2MDEW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
# 1) Wide → long
df_long = df_all.melt(
    id_vars="variable",
    var_name="datetime",
    value_name="value"
)

# 2) Convert datetime strings like "2023010100" → real timestamps
df_long["datetime"] = pd.to_datetime(df_long["datetime"], format="%Y%m%d%H")


In [41]:
na_counts = df_long.groupby("variable")["value"].apply(lambda x: x.isna().sum())
print(na_counts)


variable
ALLSKY_KT              13848
ALLSKY_SFC_PAR_TOT     17544
ALLSKY_SFC_UV_INDEX    13848
ALLSKY_SRF_ALB         13848
PRECTOTCORR            13848
PS                     13848
QV2M                   17544
RH2M                   13872
SZA                    13872
T2MDEW                 13848
T2MWET                 17544
WD10M                  13848
WS10M                  17544
Name: value, dtype: int64


In [42]:
df_long.to_pickle('df_berlin_pv.pkl')